In [32]:
from pycparser import parse_file, c_ast

global parents, variables
parents={}

In [33]:
class Condition:
    def __init__(self, condition):
        self.condition = condition
        (self.left_operand, self.operator, self.right_operand) = self.handleCondition(condition)
    
    def handleCondition(self, condition):
        left_operand = condition.left
        right_operand = condition.right
        operator = condition.op
        
        if operator == ">":
            return (right_operand, "<", left_operand)
        elif operator == ">=":
            return (right_operand, "<=", left_operand)
        else:
            return (left_operand, operator, right_operand)

class Context:  
    def __init__(self, condition = None, bool_condition = True, next_nodes = [], previous_context = None):
        if previous_context is not None:
            self.conditions = previous_context.conditions.append((condition, bool_condition))
            self.variables = previous_context.variables
            self.nexts = previous_context.nexts + next_nodes
            self.checkCondition()
        else:
            self.conditions = [] # True si la condition a été vérifié, false sinon: permet de ne pas avoir de false positives
            self.variables = {}
            self.nexts = []
    
    def addVariable(self, node_decl):
        name = node_decl.name
        types = node_decl.type.type.names
        if 'int' not in types:
            raise 'Only int type allowed'
        if 'unsigned' in types:
            return
            
    def pop(self):
        if self.nexts != []:
            return self.nexts.pop()
        return None
        
    def checkCondition(self):
        """
        Si self.state = True condition possible
        Si self.state = False condition impossible donc le branchement est inutile
        """
        
        # simplification des deux listes représentant les variables
        # propagation des singletons grâce à leur valeur trouvée dans la bibliothèque
        # si il y a encore des variables entièrement non définies de chaque côté -> deux contexte et pas d'update
        # si non définie d'un seul côté -> on a les deux contextes et update de la valeur de la variable
        # sinon -> un seul contexte et pas d'update
        self.state = True
        return
    
class Interval():
    """
        Représente un interval
        
        Attributs
        ----------
        min: @Variable 
            Minimum de l'intervalle
        max: @Variable 
            Maximum de l'intervalle
        isSingleton: Boolean
            True si c'est un singleton
    """
    def __init__(self, minimum, maximum, isSingleton):
        self.isSingleton = isSingleton
        self.min = minimum
        self.max = maximum

class Variable():
    """
        Représente une variable
        
        Attributs
        ----------
        name: String 
            Nom de la varialbe
        intervals: list[@Interval] 
            Ensemble d'intervalles pour la variable
    """
    def __init__(self, name, intervals):
        self.name = name
        self.intervals = interval

class Valeur():
    """
        Représente une valeur de variable qui est sous la forme: @Variable "operateur" Constante
        
        Attributs
        ----------
        constante: Int
            Constante
        variable: @Variable
            Variable
        op: String
            Opérateur
    """
    def __init__(self, constante = 0, variable = None, operator = "="):
        self.variable = variable
        self.operator = operator
        self.constante = constante
        
class Operation():
    def __init__(self, left, operator, right):
        self.left = left
        self.operator = operator
        self.right = right
    
    def compute(self):
        if(operator == "+"):
            if left
                    

IndentationError: expected an indented block (<ipython-input-33-2b4c689d560a>, line 33)

In [ ]:
def parse_c_file(file):
    ast = parse_file(file, use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return ast

In [ ]:
ast_graph = parse_c_file('./C files/if.c')

In [ ]:
def computeParents(node, parent):
    global parents
    parents[node] = parent
    for (child, value) in node.children():
        computeParents(value, node) 

In [ ]:
computeParents(ast_graph, None)

In [ ]:
def isIf(node):
    return type(node) == c_ast.If

In [ ]:
def getCoord(node):
    return node.coord

In [ ]:
def isDeclaration(node):
    return type(node) == c_ast.Decl

In [ ]:
def isAssignment(node):
    return type(node) == c_ast.Assignment

In [ ]:
assignments = findAssignments(ast_graph)

In [ ]:
def visitNode(context):
    node = context.pop()
    if node is not None:
        if isIf(node):
            condition = Condition(node.cond)
            context1 = Context(condition = condition, bool_condition = True, next_nodes = [node.iftrue], previous_context = context)
            context2 = Context(condition = condition, bool_condition = False, next_nodes = [node.iffalse], previous_context = context)
            if context1.state and context2.state:
                return visitNode(context1) + visitNode(context2)
            elif context1.state and not context2.state:
                return visitNode(context1) + [context2]
            elif not context1.state and context2.state:
                return visitNode(context2) + [context1]
            else:
                return [context1, context2]
        elif isDeclaration(node):
            context.addVariable(node)
        elif isAssignment(node):
            context.assign(node)
    return []

In [ ]:
def visitGraph(graph):
    context = Context(next_nodes = graph.children())
    return visitNode(context)